In [1]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
from scipy.stats import norm
df=pd.DataFrame()
df['tid']=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10']
df['itemset'] = ['CE','CEF','BE','ABCDF','AC','ACEF','ABC','AEF','ABCDE','DEF']
data=df['itemset']

 1.1 

In [2]:
def printSet(itemSets,minSupport):
    if itemSets != {}:
        df=pd.DataFrame()
        df['itemset']=[list(item) for item in itemSets.keys()]
        df['support']=itemSets.values()
        df['length'] = df['itemset'].apply(lambda x: len(x))
        df[''] = [False if val/10<minSupport else True for val in itemSets.values()]
        print(df)


In [3]:
from collections import defaultdict

def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
    combItem = {}
    for item in itemSet:
        for transaction in transactionList:
            if item.issubset(transaction):
                freqSet[item] += 1
                combItem.setdefault(item, 0)
                combItem[item] += 1
    printSet(combItem,minSupport)
    num_items=10
    combItem = {item:val for item,val in combItem.items() if val/num_items >= minSupport}
    return combItem


def joinSet(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def getItemSetTransactionList(dataset):
    transactionList = list( frozenset(transaction) for transaction in dataset)
    itemSet = set(frozenset([item]) for transaction in dataset for item in transaction)
    return itemSet, transactionList


def runApriori(data_iter, minSupport):
    itemSet, transactionList = getItemSetTransactionList(data_iter)
    print('Unique elements:',[list(item) for item in itemSet])
    freqSet = defaultdict(int)
    resultSet = {}
    print('\nStep 1')
    currentLSet = returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet)
    resultSet.update(currentLSet)
    currentLSet = set(currentLSet.keys())
    k = 2
    while currentLSet != set([]): 
        currentLSet = joinSet(currentLSet, k)
        print('\nStep %d'% k)
        currentCSet = returnItemsWithMinSupport(currentLSet, transactionList, minSupport, freqSet)
        resultSet.update(currentCSet)
        currentLSet = set(currentCSet)
        k = k + 1
    return resultSet

In [4]:
print('Data:', list(df['itemset']))
print('Minsup ratio:',5/10)
print('Minsup value:',5)
result = runApriori(df['itemset'],0.5)
print('\nResult:')
printSet(result,0.5)

Data: ['CE', 'CEF', 'BE', 'ABCDF', 'AC', 'ACEF', 'ABC', 'AEF', 'ABCDE', 'DEF']
Minsup ratio: 0.5
Minsup value: 5
Unique elements: [['C'], ['D'], ['F'], ['E'], ['A'], ['B']]

Step 1
  itemset  support  length       
0     [C]        7       1   True
1     [D]        3       1  False
2     [F]        5       1   True
3     [E]        7       1   True
4     [A]        6       1   True
5     [B]        4       1  False

Step 2
  itemset  support  length       
0  [F, A]        3       2  False
1  [F, E]        4       2  False
2  [C, E]        4       2  False
3  [A, C]        5       2   True
4  [A, E]        3       2  False
5  [F, C]        3       2  False

Step 3

Result:
  itemset  support  length      
0     [C]        7       1  True
1     [F]        5       1  True
2     [E]        7       1  True
3     [A]        6       1  True
4  [A, C]        5       2  True


1.2.

In [5]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(df['itemset']).transform(df['itemset'])
df = pd.DataFrame(te_ary, columns=te.columns_)

In [6]:
from mlxtend.frequent_patterns import apriori
results = apriori(df, min_support=0.5, use_colnames=True)
results['length'] = results['itemsets'].apply(lambda x: len(x))
print(results)

   support itemsets  length
0      0.6      (A)       1
1      0.7      (C)       1
2      0.7      (E)       1
3      0.5      (F)       1
4      0.5   (A, C)       2


1.3

In [7]:
min_sup=0.5
F={}
class FPNode(object):
    def __init__(self,names,frequency=1):
        self.name=names
        self.frequency=frequency
        self.ancestors=None
        self.children={}
        self.new=True
    def changeFreq(self):
        self.frequency+=1
    def addNode(self, node):
        if not node.name in node.children:
            self.children[node.name] = node
            node.ancestors = self
    def findNode(self, item):
        try:
            return self.children[item]
        except KeyError:
            return None
            
    def showTree(self, index=1):
        if self.name is not None:
            if self.ancestors and self.ancestors.new==True:
                print('(',end='')
                self.ancestors.new=False
            print('%s:%d:'%(self.name,self.frequency),end='')
            parent = self.ancestors
            p = ''
            while parent is not None and parent.name is not None:
                p = p + parent.name
                parent = parent.ancestors
            print(p[::-1],end='')
            if self.children == {}:
                print('(',end='')
        for child in self.children.values():
            child.showTree(index+1)
            print('),',end='')
class FPTree:
    def __init__(self):
        self.head = FPNode(None)
        self.uniq={}
        self.support_set = defaultdict(int)
    def addNode(self, transaction):
        t_node = self.head
        for item in transaction:
            self.support_set[item] +=1
            n_node = t_node.findNode(item)
            if n_node:
                n_node.changeFreq()
            else:
                n_node = FPNode(item)
                t_node.addNode(n_node)
                self.uniq.setdefault(item,[])
                self.uniq[item].append(n_node)
            t_node = n_node   
    def findNode(s_node):
        for node in self:
            if node.name == s_node:
                node.frequency+=1
    def getAncestors(self):
        for current_node in self:
            pass
        if current_node!=self.head:
            return current_node.ancestors+current_node.name
        return ''
    def showTree(self):
        self.head.showTree()
    def findAncetsetors(self,item):
        if self.name==item:
            return 
    def findSuffixPrefix(self):
        for item,arr_path in self.uniq.items():
            prefix={}
            for path in arr_path:
                node = path
                item=node.name
                frequency=node.frequency
                name_path=''
                node=node.ancestors
                prefix.setdefault(item,[])
                while node is not None and node.name is not None:
                    name_path+=node.name
                    node=node.ancestors
                if name_path!='':
                    prefix[item].append((name_path[::-1],frequency))
            print('sufix: %s, prefix: %s, empty prefix list: %s'%(item,prefix[item],True if prefix[item]== [] else False))
            makeCPTree(prefix[item],item)
            
    def findPrefixWithMinsup(self,suffix):
        for item,node in self.uniq.items():
            support=self.support_set[item]
            if support/10>=min_sup:
                return(item+suffix,support)
             

In [8]:
def getSortingUnique(dataset):
    itemSet = set(item for transaction in dataset for item in transaction)
    combItem = {}
    for item in itemSet:
        for transaction in dataset:
            if transaction.find(item)>-1:
                combItem.setdefault(item, 0)
                combItem[item] += 1
    sorted_dict={}
    sorted_keys=sorted(combItem, key = combItem.get,reverse=True)
    for key in sorted_keys:
        sorted_dict[key]=combItem[key]
    return sorted_dict
def getSortingData(dataset,sorted_dict):
    sort_data=[]
    for transaction in dataset: 
        sort_trans=[]
        for item in sorted_dict.keys():
             if transaction.find(item)>-1:
                    sort_trans.append(item)
        sort_data.append(sort_trans)
    return sort_data
def makeCPTree(prefix_list,suffix):
    tree=FPTree()
    node_sup={}
    for prefix in prefix_list: 
        path=prefix[0]
        frequency=prefix[1]
        for i in range(frequency):
            tree.addNode(path)
    node_sup.setdefault(suffix,[])
    node_sup[suffix].append(tree.findPrefixWithMinsup(suffix))
    print('CP-Tree:')
    tree.showTree()
    print('\n')
    if node_sup[suffix][0] is not None:
        name=node_sup[suffix][0][0]
        F.setdefault(name,0)
        F[name]=node_sup[suffix][0][1]
        print('Add to F: ',(name,F[name]))
    print('\n')
        
def runFPGrowth(dataset,min_support):
    data = getSortingUnique(dataset)
    print('Result sorting unique:',data)
    for item,val in data.items():
        if val/10>=min_support:
            F.setdefault(item,0)
            F[item]=val
            print('Add to F:',(item,val))
    data=getSortingData(dataset,data)
    print('Result sorting data', data)
    tree=FPTree()
    for transaction in data:
        tree.addNode(transaction)
    print('FP-Tree:')
    tree.showTree()
    print(')\n')
    tree.findSuffixPrefix()
    print('Result:')
    printSet(F,min_sup)

In [9]:
runFPGrowth(data,min_sup)

Result sorting unique: {'C': 7, 'E': 7, 'A': 6, 'F': 5, 'B': 4, 'D': 3}
Add to F: ('C', 7)
Add to F: ('E', 7)
Add to F: ('A', 6)
Add to F: ('F', 5)
Result sorting data [['C', 'E'], ['C', 'E', 'F'], ['E', 'B'], ['C', 'A', 'F', 'B', 'D'], ['C', 'A'], ['C', 'E', 'A', 'F'], ['C', 'A', 'B'], ['E', 'A', 'F'], ['C', 'E', 'A', 'B', 'D'], ['E', 'F', 'D']]
FP-Tree:
(C:7:(E:4:C(F:1:CE(),A:2:CE(F:1:CEA(),B:1:CEA(D:1:CEAB(),),),),A:3:C(F:1:CA(B:1:CAF(D:1:CAFB(),),),B:1:CA(),),),E:3:(B:1:E(),A:1:E(F:1:EA(),),F:1:E(D:1:EF(),),),)

sufix: C, prefix: [], empty prefix list: True
CP-Tree:




sufix: E, prefix: [('C', 4)], empty prefix list: False
CP-Tree:
(C:4:(),



sufix: F, prefix: [('CE', 1), ('CA', 1), ('CEA', 1), ('EA', 1), ('E', 1)], empty prefix list: False
CP-Tree:
(C:3:(E:2:C(A:1:CE(),),A:1:C(),),E:2:(A:1:E(),),



sufix: B, prefix: [('E', 1), ('CAF', 1), ('CA', 1), ('CEA', 1)], empty prefix list: False
CP-Tree:
(E:1:(),C:3:(A:2:C(F:1:CA(),),E:1:C(A:1:CE(),),),



sufix: A, prefix: [('C', 3), (

1.4

In [10]:
from mlxtend.frequent_patterns import fpgrowth
results = fpgrowth(df, min_support=0.5, use_colnames=True)
results['length'] = results['itemsets'].apply(lambda x: len(x))
print(results)

   support itemsets  length
0      0.7      (E)       1
1      0.7      (C)       1
2      0.5      (F)       1
3      0.6      (A)       1
4      0.5   (A, C)       2


### Задание 2

In [11]:
dataset=['CDHJ','ABDEFHJ','AEGJ','ACEFG','CFGI','CDEHIJ','BEFGJ','ADHI','ABF','DHI']

In [12]:
print('Data:', dataset)
print('Minsup ratio:',3/10)
print('Minsup value:',3)

Data: ['CDHJ', 'ABDEFHJ', 'AEGJ', 'ACEFG', 'CFGI', 'CDEHIJ', 'BEFGJ', 'ADHI', 'ABF', 'DHI']
Minsup ratio: 0.3
Minsup value: 3


2.1

In [13]:
itemSet,_ = getItemSetTransactionList(dataset)
print('Simple set count:',len(itemSet))

Simple set count: 10


2.2

In [14]:
symbol_map={'A':'O','M':'O','N':'O','I':'Q','P':'Q','G':'G','H':'H','A':'A','B':'M','C':'M','E':'N','D':'N', 'F':'N','I':'I',
            'J':'P','K':'P','L':'P',
            'B':'B','C':'C','D':'D','E':'E','F':'F','J':'J','K':'K','L':'L'}
levels=5
while(levels>1):
    new_data=[]
    for transaction in dataset:
        str=''
        for item, val in symbol_map.items():
            if transaction.find(item)>-1:
                str=str+val
        new_data.append(str)
    levels-=1
    dataset=new_data
print('Symbol map:',symbol_map)
print('R data:',new_data)

result = runApriori(new_data,0.5)
print('\nResult:')
printSet(result,0.5)



Symbol map: {'A': 'A', 'M': 'O', 'N': 'O', 'I': 'I', 'P': 'Q', 'G': 'G', 'H': 'H', 'B': 'B', 'C': 'C', 'E': 'E', 'D': 'D', 'F': 'F', 'J': 'J', 'K': 'K', 'L': 'L'}
R data: ['HCDJ', 'AHBEDFJ', 'AGEJ', 'AGCEF', 'IGCF', 'IHCEDJ', 'GBEFJ', 'AIHD', 'ABF', 'IHD']
Unique elements: [['C'], ['G'], ['D'], ['F'], ['H'], ['E'], ['A'], ['J'], ['B'], ['I']]

Step 1
  itemset  support  length       
0     [C]        4       1  False
1     [G]        4       1  False
2     [D]        5       1   True
3     [F]        5       1   True
4     [H]        5       1   True
5     [E]        5       1   True
6     [A]        5       1   True
7     [J]        5       1   True
8     [B]        3       1  False
9     [I]        4       1  False

Step 2
   itemset  support  length       
0   [H, D]        5       2   True
1   [F, A]        3       2  False
2   [J, F]        2       2  False
3   [F, E]        3       2  False
4   [J, D]        3       2  False
5   [J, A]        2       2  False
6   [D, A]        2 